In [1]:
from time import perf_counter

from policy_core.cost import CostModel, set_exp_refund_ratio
from policy_core.data import BUFF_TYPE_COUNTS, BUFF_TYPE_MAX_VALUES, BUFF_TYPES
from policy_core.scoring import (
    build_score_pmfs_from_counts,
    create_buff_weight_list,
    make_linear_score_fn,
    make_simple_score_fn,
)
from policy_core.simulation import (
    always_upgrade,
    simulate_many,
    strategy_two_one_two,
    upgrade_until_impossible,
)
from policy_core.solver import PolicySolver


In [2]:
# Adjust the EXP refund ratio used by the cost helpers.
set_exp_refund_ratio(0.66)


In [67]:
PRIMARY_BUFF_WEIGHTS = {
    "Crit_Rate": 122,
    "Crit_Damage": 100,
    "Attack": 0,
    "Defence": 0,
    "HP": 52,
    "Attack_Flat": 0,
    "Defence_Flat": 0,
    "HP_Flat": 18,
    "ER": 0,
    "Basic_Attack_Damage": 40,
    "Heavy_Attack_Damage": 0,
    "Skill_Damage": 7,
    "Ult_Damage": 30,
}

primary_weight_list = create_buff_weight_list(PRIMARY_BUFF_WEIGHTS)
primary_scorer = make_linear_score_fn(primary_weight_list)
primary_pmfs = build_score_pmfs_from_counts(BUFF_TYPE_COUNTS, primary_scorer)

TARGET_SCORE_PRIMARY = 10.0

cost_model_primary = CostModel(w_echo=0.0, w_dkq=1.0, w_exp=0.0)
succ_extra_cost_primary = cost_model_primary.succ_extra_cost()
solver_primary = PolicySolver(
    pmfs=primary_pmfs,
    target=TARGET_SCORE_PRIMARY,
    cost_model=cost_model_primary,
    raw_counts=[dict(counts) for counts in BUFF_TYPE_COUNTS],
    scorer_fn=primary_scorer,
)


In [ ]:
start = perf_counter()
lambda_star_primary = solver_primary.lambda_search()
elapsed_primary = perf_counter() - start

expected_cost_primary = (
    float("inf") if lambda_star_primary <= 0 else (1.0 / lambda_star_primary + succ_extra_cost_primary)
)

print(f"λ* ≈ {lambda_star_primary:.8f}")
print(f"Expected cost per successful echo = {expected_cost_primary:.2f}")
print(f"(lambda_search runtime: {elapsed_primary:.3f}s)")

solver_primary.derive_policy_at_lambda(lambda_star_primary)

success_rate_primary, echo_per_success_primary, dkq_per_success_primary, exp_per_success_primary, _, _ = simulate_many(
    solver=solver_primary,
    runs=1_000_000,
    seed=42,
)
print("\nMonte Carlo (policy at λ*):")
print(f"  P(success rate per attempt) ≈ {100 * success_rate_primary:.4f}%")
print(f"  胚子消耗 per success ≈ {echo_per_success_primary:.2f}")
print(f"  调谐器消耗 per success ≈ {dkq_per_success_primary:.2f}")
print(f"  密音筒 per success ≈ {exp_per_success_primary:.2f}")
print(f"  EXP per success ≈ {5 * exp_per_success_primary:.2f}k")

λ* ≈ 0.02590111
Expected cost per successful echo = 53.61
(lambda_search runtime: 0.806s)

Monte Carlo (policy at λ*):
  P(success rate per attempt) ≈ 90.6561%
  胚子消耗 per success ≈ 1.10
  调谐器消耗 per success ≈ 53.61
  密音筒 per success ≈ 29.60
  EXP per success ≈ 148.01k


In [7]:
runs = 100000

success_rate_always, echo_per_success_always, dkq_per_success_always, exp_per_success_always, max_slot_scores, _ = simulate_many(
    solver=solver_primary,
    runs=runs,
    seed=114514,
    decision=always_upgrade,
)
print("\nMonte Carlo (always upgrade to max):")
print(f"  P(success rate per attempt) ≈ {100 * success_rate_always:.4f}%")
print(f"  胚子消耗 per success ≈ {echo_per_success_always:.2f}")
print(f"  调谐器消耗 per success ≈ {dkq_per_success_always:.2f}")
print(f"  密音筒 per success ≈ {exp_per_success_always:.2f}")
print(f"  平均分数 ≈ {sum(max_slot_scores)/len(max_slot_scores)/100:.2f}")


Monte Carlo (always upgrade to max):
  P(success rate per attempt) ≈ 49.9850%
  胚子消耗 per success ≈ 2.00
  调谐器消耗 per success ≈ 85.02
  密音筒 per success ≈ 38.33
  平均分数 ≈ 30.20


In [9]:
runs = 100000

success_rate_always, echo_per_success_always, dkq_per_success_always, exp_per_success_always, _, _ = simulate_many(
    solver=solver_primary,
    runs=runs,
    seed=114514,
    decision=always_upgrade,
)
print("\nMonte Carlo (always upgrade to max):")
print(f"  P(success rate per attempt) ≈ {100 * success_rate_always:.4f}%")
print(f"  胚子消耗 per success ≈ {echo_per_success_always:.2f}")
print(f"  调谐器消耗 per success ≈ {dkq_per_success_always:.2f}")
print(f"  密音筒 per success ≈ {exp_per_success_always:.2f}")

decision_until_impossible = upgrade_until_impossible(solver_primary)
success_rate_until_impossible, echo_per_success_until_impossible, dkq_per_success_until_impossible, exp_per_success_until_impossible, _, _ = simulate_many(
    solver=solver_primary,
    runs=runs,
    seed=114514,
    decision=decision_until_impossible,
)
print("\nMonte Carlo (upgrade until impossible):")
print(f"  P(success rate per attempt) ≈ {100 * success_rate_until_impossible:.4f}%")
print(f"  胚子消耗 per success ≈ {echo_per_success_until_impossible:.2f}")
print(f"  调谐器消耗 per success ≈ {dkq_per_success_until_impossible:.2f}")
print(f"  密音筒 per success ≈ {exp_per_success_until_impossible:.2f}")

success_rate_two_one_two, echo_per_success_two_one_two, dkq_per_success_two_one_two, exp_per_success_two_one_two, _, _ = simulate_many(
    solver=solver_primary,
    runs=runs,
    seed=114514,
    decision=strategy_two_one_two,
)
print("\nMonte Carlo (2-1-2 strategy):")
print(f"  P(success rate per attempt) ≈ {100 * success_rate_two_one_two:.4f}%")
print(f"  胚子消耗 per success ≈ {echo_per_success_two_one_two:.2f}")
print(f"  调谐器消耗 per success ≈ {dkq_per_success_two_one_two:.2f}")
print(f"  密音筒 per success ≈ {exp_per_success_two_one_two:.2f}")



Monte Carlo (always upgrade to max):
  P(success rate per attempt) ≈ 0.6130%
  胚子消耗 per success ≈ 163.13
  调谐器消耗 per success ≈ 5724.62
  密音筒 per success ≈ 1184.52

Monte Carlo (upgrade until impossible):
  P(success rate per attempt) ≈ 0.6260%
  胚子消耗 per success ≈ 159.74
  调谐器消耗 per success ≈ 4304.12
  密音筒 per success ≈ 625.82

Monte Carlo (2-1-2 strategy):
  P(success rate per attempt) ≈ 0.4350%
  胚子消耗 per success ≈ 229.89
  调谐器消耗 per success ≈ 3533.78
  密音筒 per success ≈ 468.79


In [19]:
BUFF_WEIGHT_SIMPLE = {
    "Crit_Rate": 3,
    "Crit_Damage": 3,
    "Attack": 0,
    "Defence": 0,
    "HP": 1,
    "Attack_Flat": 0,
    "Defence_Flat": 0,
    "HP_Flat": 1,
    "ER": 0,
    "Basic_Attack_Damage": 1,
    "Heavy_Attack_Damage": 0,
    "Skill_Damage": 0,
    "Ult_Damage": 0,
}

buff_weight_simple_list = create_buff_weight_list(BUFF_WEIGHT_SIMPLE)
simple_scorer = make_simple_score_fn(buff_weight_simple_list)
simple_pmfs = build_score_pmfs_from_counts(BUFF_TYPE_COUNTS, simple_scorer)

TARGET_SCORE_SIMPLE = 7

cost_model_simple = CostModel(w_echo=0.0, w_dkq=1.0, w_exp=0.0)
succ_extra_cost_simple = cost_model_simple.succ_extra_cost()
solver_simple = PolicySolver(
    pmfs=simple_pmfs,
    target=TARGET_SCORE_SIMPLE,
    cost_model=cost_model_simple,
    raw_counts=[dict(counts) for counts in BUFF_TYPE_COUNTS],
    scorer_fn=simple_scorer,
)

start_simple = perf_counter()
lambda_star_simple = solver_simple.lambda_search()
elapsed_simple = perf_counter() - start_simple

expected_cost_simple = (
    float("inf")
    if lambda_star_simple <= 0
    else (1.0 / lambda_star_simple + succ_extra_cost_simple)
)

print(f"λ* ≈ {lambda_star_simple:.8f}")
print(f"Expected cost per successful echo = {expected_cost_simple:.2f}")
print(f"(lambda_search runtime: {elapsed_simple:.3f}s)")

solver_simple.derive_policy_at_lambda(lambda_star_simple)

success_rate_simple, echo_per_success_simple, dkq_per_success_simple, exp_per_success_simple, _, _ = simulate_many(
    solver=solver_simple,
    runs=5000000,
    seed=114514,
)
print("\nMonte Carlo (policy at λ*):")
print(f"  P(success rate per attempt) ≈ {100 * success_rate_simple:.4f}%")
print(f"  胚子消耗 per success ≈ {echo_per_success_simple:.2f}")
print(f"  调谐器消耗 per success ≈ {dkq_per_success_simple:.2f}")
print(f"  密音筒 per success ≈ {exp_per_success_simple:.2f}")
print(f"  EXP per success ≈ {5 * exp_per_success_simple:.2f}k")


λ* ≈ 0.00335851
Expected cost per successful echo = 312.75
(lambda_search runtime: 0.034s)

Monte Carlo (policy at λ*):
  P(success rate per attempt) ≈ 5.1506%
  胚子消耗 per success ≈ 19.42
  调谐器消耗 per success ≈ 312.01
  密音筒 per success ≈ 61.59
  EXP per success ≈ 307.95k


In [12]:
runs = 1000000

success_rate_always, echo_per_success_always, dkq_per_success_always, exp_per_success_always, _, _ = simulate_many(
    solver=solver_simple,
    runs=runs,
    seed=114514,
    decision=always_upgrade,
)
print("\nMonte Carlo (always upgrade to max):")
print(f"  P(success rate per attempt) ≈ {100 * success_rate_always:.4f}%")
print(f"  胚子消耗 per success ≈ {echo_per_success_always:.2f}")
print(f"  调谐器消耗 per success ≈ {dkq_per_success_always:.2f}")
print(f"  密音筒 per success ≈ {exp_per_success_always:.2f}")

decision_until_impossible = upgrade_until_impossible(solver_simple)
success_rate_until_impossible, echo_per_success_until_impossible, dkq_per_success_until_impossible, exp_per_success_until_impossible, _, _ = simulate_many(
    solver=solver_simple,
    runs=runs,
    seed=114514,
    decision=decision_until_impossible,
)
print("\nMonte Carlo (upgrade until impossible):")
print(f"  P(success rate per attempt) ≈ {100 * success_rate_until_impossible:.4f}%")
print(f"  胚子消耗 per success ≈ {echo_per_success_until_impossible:.2f}")
print(f"  调谐器消耗 per success ≈ {dkq_per_success_until_impossible:.2f}")
print(f"  密音筒 per success ≈ {exp_per_success_until_impossible:.2f}")

success_rate_two_one_two, echo_per_success_two_one_two, dkq_per_success_two_one_two, exp_per_success_two_one_two, _, _ = simulate_many(
    solver=solver_simple,
    runs=runs,
    seed=114514,
    decision=strategy_two_one_two,
)
print("\nMonte Carlo (2-1-2 strategy):")
print(f"  P(success rate per attempt) ≈ {100 * success_rate_two_one_two:.4f}%")
print(f"  胚子消耗 per success ≈ {echo_per_success_two_one_two:.2f}")
print(f"  调谐器消耗 per success ≈ {dkq_per_success_two_one_two:.2f}")
print(f"  密音筒 per success ≈ {exp_per_success_two_one_two:.2f}")



Monte Carlo (always upgrade to max):
  P(success rate per attempt) ≈ 8.4675%
  胚子消耗 per success ≈ 11.81
  调谐器消耗 per success ≈ 428.35
  密音筒 per success ≈ 105.59

Monte Carlo (upgrade until impossible):
  P(success rate per attempt) ≈ 8.4866%
  胚子消耗 per success ≈ 11.78
  调谐器消耗 per success ≈ 360.34
  密音筒 per success ≈ 77.74

Monte Carlo (2-1-2 strategy):
  P(success rate per attempt) ≈ 5.1923%
  胚子消耗 per success ≈ 19.26
  调谐器消耗 per success ≈ 309.84
  密音筒 per success ≈ 58.91


In [ ]:
# ----------------------------
# Compare with ww-toolbox
# ----------------------------

#convert weights

for idx in (0, 1, 2, 3, 4, 8, 9, 10, 11, 12):
    scaled = primary_weight_list[idx] / BUFF_TYPE_MAX_VALUES[idx] * 10
    print(BUFF_TYPES[idx], scaled)
for idx in (5, 6, 7):
    scaled = primary_weight_list[idx] / BUFF_TYPE_MAX_VALUES[idx]
    print(BUFF_TYPES[idx], scaled)

print(primary_scorer(0, 87) + primary_scorer(1, 174) + primary_scorer(4, 86) + primary_scorer(11, 86) + primary_scorer(12, 94))


In [ ]:
PRIMARY_BUFF_WEIGHTS = {
    "Crit_Rate": 63,
    "Crit_Damage": 63,
    "Attack": 46.4,
    "Defence": 0,
    "HP": 0,
    "Attack_Flat": 25,
    "Defence_Flat": 0,
    "HP_Flat": 0,
    "ER": 6.2,
    "Basic_Attack_Damage": 0,
    "Heavy_Attack_Damage": 0,
    "Skill_Damage": 29,
    "Ult_Damage": 5.8,
}

primary_weight_list = create_buff_weight_list(PRIMARY_BUFF_WEIGHTS)
primary_scorer = make_linear_score_fn(primary_weight_list)
primary_pmfs = build_score_pmfs_from_counts(BUFF_TYPE_COUNTS, primary_scorer)

TARGET_SCORE_PRIMARY = 10.0

cost_model_primary = CostModel(w_echo=0.0, w_dkq=1.0, w_exp=0.0)
succ_extra_cost_primary = cost_model_primary.succ_extra_cost()
solver_primary = PolicySolver(
    pmfs=primary_pmfs,
    target=TARGET_SCORE_PRIMARY,
    cost_model=cost_model_primary,
    raw_counts=[dict(counts) for counts in BUFF_TYPE_COUNTS],
    scorer_fn=primary_scorer,
)


In [ ]:
start = perf_counter()
lambda_star_primary = solver_primary.lambda_search()
elapsed_primary = perf_counter() - start

expected_cost_primary = (
    float("inf") if lambda_star_primary <= 0 else (1.0 / lambda_star_primary + succ_extra_cost_primary)
)

print(f"λ* ≈ {lambda_star_primary:.8f}")
print(f"Expected cost per successful echo = {expected_cost_primary:.2f}")
print(f"(lambda_search runtime: {elapsed_primary:.3f}s)")

solver_primary.derive_policy_at_lambda(lambda_star_primary)

success_rate_primary, echo_per_success_primary, dkq_per_success_primary, exp_per_success_primary, _, _ = simulate_many(
    solver=solver_primary,
    runs=1_000_000,
    seed=42,
)
print("\nMonte Carlo (policy at λ*):")
print(f"  P(success rate per attempt) ≈ {100 * success_rate_primary:.4f}%")
print(f"  胚子消耗 per success ≈ {echo_per_success_primary:.2f}")
print(f"  调谐器消耗 per success ≈ {dkq_per_success_primary:.2f}")
print(f"  密音筒 per success ≈ {exp_per_success_primary:.2f}")
print(f"  EXP per success ≈ {5 * exp_per_success_primary:.2f}k")